In [1]:
#TODO: in 'http response information' and below, I changed the name of the query, 
#      but this is platform dependent, I think?
#      In 'customise or replace dataframe conversions' and below, the query doesn't work:
#      'Can't specify two absolute points and a relative duration'


#Use query for weather data

In [2]:
from waylay import WaylayClient, RestResponseError
waylay_client = WaylayClient.from_profile('staging')

# Waylay Python SDK - Making REST calls

The Waylay Python SDK exposes a number of _REST Services_, each a collection of _REST Resources_ that have _action methods_.

For an overview, see 
> `[IO]` https://docs-io.waylay.io/#/api/sdk/python<br>
> `[Enterprise]` https://docs.waylay.io/api/sdk/python/

In most cases these _action methods_ directly correspond to an underlying REST endpoint of the waylay platform.

In general, the Python SDK takes care of
* authentication
* the endpoint url to call, and HTTP method to use
* converting JSON responses to Python objects, extracting the relevant data
* converting Python request body objects to JSON
* handling errors 

Check out the documentation sites ([io](https://docs-io.waylay.io/#/api/sdk/python), [enterprise](https://docs.waylay.io/api/sdk/python/)) on

* how requests and response are exactly represented as json
* which additional parameters can be provided
* what the preconditions and effects are of your call

### request arguments
These are the general rules for making requests, corresponding to the REST documentation of the underlying REST action:
* url **path parameters** are passed as **positional arguments**:
  ###### example

  > `waylay_client.queries.query.get('151CF-temperature')` 

  will use 151CF-temperature to construct a request to fetch this query definition

  > `GET .../config/query/151CF-temperature` 

* **query parameters** belong in a named **`params` argument**:
  ###### example
  > `waylay_client.queries.query.data('151CF-temperature', params={'from':'2021-03-01T10:00:00+00:00'})`
  
  will bind the `from` parameter

* **request objects** are passed into a **`body` argument**:
  ###### example
  > `waylay_client.queries.query.execute(body={'resource':'RDJ_89839','metric':'revolutions'})`}|

### response handling
These are the general rules on how the SDK treats results of REST calls:
* Only successfull responses (HTTP status `2XX`) return a result, other responses raise an exception.
* The SDK extracts the most relevant part of the (json) response body. You can use the `select_path` argument to override this behaviour.
* When the REST call produces **timeseries data**, the SDK will return a **pandas DataFrame**.
* By providing the  `raw=True` argument, you instruct the SDK to skip all error and response handling, and **just return the HTTP response**.

### error handling

The SDK uses the following exception hierarchy to notify problems. These exception classes belong to the `waylay.exception` module. They all descend from a `WaylayError` base class.

| exception class | raised in case of |
| --------------- | ----------- |
| `AuthError` | Waylay authentication errors. |
| `ConfigError` |  Waylay client configuration errors. |
| `RequestError` | Errors in tools and utilities that are not directly related to a REST call. |
| `RestRequestError` | Failure to prepare a REST call. |
| `RestResponseError` | Wraps the result of a failed REST call. |
| `RestResponseParseError` | Failure to parse the result of a succesfull REST call. |

Errors of type `RestResponse` have the following attributes that you can use to handle problems:

* `response` contains the full HTTP Response object of the REST call, which lets you inspect the status code (`response.status_code`) , response body (`response.body`) and other attributes such as _headers_.
* `message` will give you the most relevant error message




### http response information

Most _action methods_ support a `raw=True` parameter. This will prevent exception handling and parsing of the REST call by the SDK. The unparsed result and http response information is returned in a _Response_ object with attributes
 * `body` : the result data (json and csv data is parsed to python data structures)
 * standard http information such as `url`, `method`, `headers`, `status_code`, `client_response`

In [3]:
waylay_client.queries.query.list()

['151CF multiple aggregations',
 '151D8 space',
 'arch',
 'battery-life-testset2',
 'battery-life-testset-sdk0.3',
 'bert-666',
 'bert_test',
 'boo',
 'boo2',
 'byoml-query']

In [4]:
http_response = waylay_client.queries.query.get('weather-data-query', raw=True)
{
    'url':http_response.url,
    'method':http_response.method,
    'content-type':http_response.headers['content-type'],
    'status_code':http_response.status_code,
    'name':http_response.body['name'],
}


{'url': 'https://api-staging.waylay.io/queries/v1/query/weather-data-query',
 'method': 'GET',
 'content-type': 'application/json',
 'status_code': 200,
 'name': 'weather-data-query'}

### access http response information from an error
When an request is unsuccessfull, the client will raise an exception. 

These exceptions (from the `waylay.exceptions` module) are either (instances of subclasses of)
* a `RestRequestError` that indicates a problem before sending an api call to waylay (e.g. when input argument conversion fails)
* a `RestResponseError` that reports a problem from or after the api call to waylay. This exception gives you access to the underlying response (`response` attribute)
  * a `RestResponseParseError` error indicates a problem in processing a succesfull response from the waylay platform. All other `RestResponseError` will come from errors reported by the waylay platform itself (http status code above the `200` range).
  
Other errors can occur (such as standard python `ValueError`,`TypeError` or `AttributeError`) but these will normally indicate a programming error. Networking failures will normally result in a `ClientConnectionError`

In [5]:
# try to get the representation of a `query` entity that does not exist. 
# this will result in a `404 NOT FOUND` error 
try:
   waylay_client.queries.query.get('where are you???')
except RestResponseError as exc:
   print(exc.message)
   print(exc.response.status_code)
   print(exc.response.url)
    


operation=not_found_error
404
https://api-staging.waylay.io/queries/v1/query/where%20are%20you


## accessing the complete response
The SDK extracts the most relevant part of the (json) response body. You can use the `select_path` argument to override this behaviour

In [6]:
# the `query` entity in the REST response is given as default response on the SDK action
waylay_client.queries.query.get('weather-data-query')

{'until': 'now',
 'from': '2022-01-01',
 'data': [{'metric': 'temperature',
   'resource': 'weather_data',
   'aggregation': 'mean'}]}

In [7]:
waylay_client.queries.query.get('weather-data-query', select_path=False)

{'_links': {'self': {'href': 'https://api-staging.waylay.io/queries/v1/query/weather-data-query'},
  'execute': {'href': 'https://api-staging.waylay.io/queries/v1/data/weather-data-query'}},
 'attrs': {'created': '2022-09-01T14:28:28.433652+00:00',
  'created_by': 'users/1b2c6668-b42a-44fe-8ca7-8170e998a9f1',
  'modified': '2022-09-01T14:28:28.433652+00:00',
  'modified_by': 'users/1b2c6668-b42a-44fe-8ca7-8170e998a9f1',
  'path': '/tsa/query/weather-data-query'},
 'name': 'weather-data-query',
 'meta': {},
 'query': {'until': 'now',
  'from': '2022-01-01',
  'data': [{'metric': 'temperature',
    'resource': 'weather_data',
    'aggregation': 'mean'}]}}

In [8]:
waylay_client.queries.query.get('weather-data-query', select_path=['attrs','modified'])

'2022-09-01T14:28:28.433652+00:00'

## customise or replace dataframe conversions
The methods of the `queries.query` that return timeseries data, will normally return their data as a pandas `DataFrame`. These have immediate display support in jupyter notebooks:

In [9]:
waylay_client.queries.query.execute(
    'weather-data-query', 
    params={'from': '2022-01-05'}
)

resource,weather_data
metric,temperature
timestamp,
2022-01-05 00:00:00+00:00,11
2022-01-05 00:51:00+00:00,11
2022-01-05 01:51:00+00:00,11
2022-01-05 02:51:00+00:00,6
2022-01-05 03:00:00+00:00,6
...,...
2022-01-08 03:00:00+00:00,17
2022-01-08 03:51:00+00:00,22


You can use the `response_constructor` parameter to replace the dataframe constructor with your own method. 

Other _action methods_ that apply conversions to raw json data structures normally support this same optional parameter.

A falsy value will let the method return just the json data payload (as python object).

In [10]:
waylay_client.queries.query.execute(
    'weather-data-query', 
    params={'from': '2022-01-05'},
    response_constructor=False
)

[{'columns': ['timestamp',
   {'resource': 'weather_data', 'metric': 'temperature'}],
  'data': [[1641340800000, 11],
   [1641343860000, 11],
   [1641347460000, 11],
   [1641351060000, 6],
   [1641351600000, 6],
   [1641354660000, 6],
   [1641358260000, 0],
   [1641358740000, 9999],
   [1641361860000, 0],
   [1641362400000, 0],
   [1641365460000, 0],
   [1641369060000, 6],
   [1641372660000, 11],
   [1641373200000, 11],
   [1641376260000, 11],
   [1641378360000, 17],
   [1641379380000, 17],
   [1641379860000, 17],
   [1641380400000, 17],
   [1641381060000, 17],
   [1641383460000, 17],
   [1641384000000, 17],
   [1641386940000, 20],
   [1641387060000, 22],
   [1641388860000, 22],
   [1641390540000, 30],
   [1641390660000, 33],
   [1641394260000, 39],
   [1641394800000, 39],
   [1641397020000, 50],
   [1641397860000, 50],
   [1641400200000, 50],
   [1641401460000, 56],
   [1641405060000, 67],
   [1641405600000, 67],
   [1641406980000, 72],
   [1641408660000, 72],
   [1641412260000, 78],


In [11]:
# return a map with timestamps as keys, the observation values as value. 
waylay_client.queries.query.execute(
    'weather-data-query', 
    params={'from': '2022-01-05'},
    response_constructor=lambda d: { row[0] : row[1:] for row in d[0]['data'] }
)

{1641340800000: [11],
 1641343860000: [11],
 1641347460000: [11],
 1641351060000: [6],
 1641351600000: [6],
 1641354660000: [6],
 1641358260000: [0],
 1641358740000: [9999],
 1641361860000: [0],
 1641362400000: [0],
 1641365460000: [0],
 1641369060000: [6],
 1641372660000: [11],
 1641373200000: [11],
 1641376260000: [11],
 1641378360000: [17],
 1641379380000: [17],
 1641379860000: [17],
 1641380400000: [17],
 1641381060000: [17],
 1641383460000: [17],
 1641384000000: [17],
 1641386940000: [20],
 1641387060000: [22],
 1641388860000: [22],
 1641390540000: [30],
 1641390660000: [33],
 1641394260000: [39],
 1641394800000: [39],
 1641397020000: [50],
 1641397860000: [50],
 1641400200000: [50],
 1641401460000: [56],
 1641405060000: [67],
 1641405600000: [67],
 1641406980000: [72],
 1641408660000: [72],
 1641412260000: [78],
 1641415140000: [83],
 1641415860000: [83],
 1641416400000: [83],
 1641418740000: [89],
 1641419460000: [89],
 1641423060000: [83],
 1641425340000: [83],
 1641426660000: 

In [12]:
import numpy as np
# return timeseries data as a numpy array, transposed as an array per series 
waylay_client.queries.query.execute(
    'weather-data-query', 
    params={'from': '2022-01-05'},
    response_constructor=lambda d: np.transpose(d[0]['data'])
)

array([[1641340800000, 1641343860000, 1641347460000, 1641351060000,
        1641351600000, 1641354660000, 1641358260000, 1641358740000,
        1641361860000, 1641362400000, 1641365460000, 1641369060000,
        1641372660000, 1641373200000, 1641376260000, 1641378360000,
        1641379380000, 1641379860000, 1641380400000, 1641381060000,
        1641383460000, 1641384000000, 1641386940000, 1641387060000,
        1641388860000, 1641390540000, 1641390660000, 1641394260000,
        1641394800000, 1641397020000, 1641397860000, 1641400200000,
        1641401460000, 1641405060000, 1641405600000, 1641406980000,
        1641408660000, 1641412260000, 1641415140000, 1641415860000,
        1641416400000, 1641418740000, 1641419460000, 1641423060000,
        1641425340000, 1641426660000, 1641427200000, 1641430260000,
        1641433860000, 1641437460000, 1641438000000, 1641441060000,
        1641444660000, 1641445140000, 1641448260000, 1641448800000,
        1641451860000, 1641455460000, 1641459060